In [15]:
import src.data.breathe_data as bd
import src.inference.long_inf_slicing as slicing
import src.models.builders as mb
import src.models.var_builders as var_builders
import src.inference.helpers as ih
from plotly.subplots import make_subplots
import src.models.helpers as mh

import pandas as pd
import numpy as np

In [16]:
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

INFO:root:* Checking for same day measurements *


## Calc consecutive days

In [17]:
def get_consec_days(df):
    df = df.sort_values(by="Date Recorded")
    df["Prev day"] = df["Date Recorded"].shift(1)
    # Compute n days between measurements
    df["Days elapsed"] = df["Date Recorded"] - df["Prev day"]
    # Remove nan
    df = df.dropna(subset=["Days elapsed"])

    # Get first idx where Days elapsed is greater than 1
    idx = df[df["Days elapsed"] > pd.Timedelta(days=3)].index
    print(idx)

    return df


df_tmp = df[df.ID == "405"]
df_tmp = get_consec_days(df_tmp)
# Not finished

Index([], dtype='int64')


In [12]:
df_tmp['Days elapsed'].value_counts()

Days elapsed
1 days    1012
2 days      22
Name: count, dtype: int64

## Inference through time

In [5]:
# Initialize variables (not used for inference)
# interconnected_AR = True
interconnected_AR = "fix message up to HFEV1 to truncated uniform"
# ar_change_cpt_suffix = "_padded_identity_15"
ar_change_cpt_suffix = "_shift_span_[-20;20]_samples"
save = True
if interconnected_AR == "fix message up to HFEV1 to truncated uniform":
    inteconnected_AR_str = "interdays AR v2 with trunc uni message to HFEV1, "
else:
    inteconnected_AR_str = ""


def infer_and_plot_for_id(df_for_ID, diff_threshold=1e-8):
    df_for_ID = df_for_ID.reset_index(drop=True)
    print(f"\nID: {df_for_ID.ID.iloc[0]}")
    print(f"Number of datapoints: {len(df_for_ID)}")

    height = df_for_ID.Height.iloc[0]
    age = df_for_ID.Age.iloc[0]
    sex = df_for_ID.Sex.iloc[0]
    (
        _,
        inf_alg,
        HFEV1,
        ecFEV1,
        AR,
        HO2Sat,
        O2SatFFA,
        IA,
        UO2Sat,
        O2Sat,
        ecFEF2575prctecFEV1,
    ) = mb.o2sat_fev1_fef2575_long_model_shared_healthy_vars_and_temporal_ar(
        height, age, sex, ia_prior="breathe", ar_change_cpt_suffix=ar_change_cpt_suffix
    )

    vars = [AR, IA]
    shared_vars = [HFEV1, HO2Sat]
    # obs_vars = [ecFEV1.name, O2Sat.name]
    # obs_vars = [ecFEV1.name, O2Sat.name, ecFEF2575prctecFEV1.name]
    # obs_vars = [ecFEV1.name, ecFEF2575prctecFEV1.name]
    obs_vars = [ecFEV1.name]

    df_query_res, df_res_before_convergence, shared_vars_final = (
        slicing.query_back_and_forth_across_days_AR(
            df_for_ID,
            inf_alg,
            shared_vars,
            vars,
            obs_vars,
            diff_threshold,
            debug=False,
            max_passes=1,
            interconnect_AR=interconnected_AR,
            auto_reset_shared_vars=False
        )
    )

    colorscale = [
        [0, "white"],
        [0.01, "red"],
        [0.05, "yellow"],
        [0.1, "cyan"],
        [0.6, "blue"],
        [1, "black"],
    ]

    obs_vars_str = ", ".join([mh.name_to_abbr(var) for var in obs_vars])
    ar_prior_str = f"AR {ar_change_cpt_suffix}"
    passes = "back and forth"

    title = f"ID {df_for_ID.ID[0]} - obs. {obs_vars_str}, {passes}, {inteconnected_AR_str}{ar_prior_str} ({df_for_ID.Sex[0]}, {df_for_ID.Age[0]}yr, {df_for_ID.Height[0]}cm)"
    slicing.plot_posterior_validation(
        df_res_before_convergence,
        HFEV1,
        AR,
        df_for_ID,
        title,
        colorscale,
        save,
    )

    title = f"ID {df_for_ID.ID[0]} - obs. {obs_vars_str}, {inteconnected_AR_str}{ar_prior_str} ({df_for_ID.Sex[0]}, {df_for_ID.Age[0]}yr, {df_for_ID.Height[0]}cm)"
    slicing.plot_query_res(
        df_for_ID,
        ecFEV1,
        O2Sat,
        df_query_res,
        AR,
        IA,
        HFEV1,
        HO2Sat,
        title,
        colorscale,
        save,
    )
    return df_query_res, df_res_before_convergence, shared_vars_final


# interesting_ids = [
#     "132",
#     "146",
#     "177",
#     "180",
#     "202",
#     "527",
#     "117",
#     "131",
#     "134",
#     "191",
#     "139",
#     "253",
#     "101",
# ]

# Consecutive entries are less than 3 days apart until
# 101     (592, 1680)

df_for_ID = df[df["ID"] == "101"].iloc[:591]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "405"]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "272"].iloc[:417]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "201"].iloc[:289]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "203"].iloc[:285]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, diff_threshold=1e-2
)
# df[df.ID.isin(interesting_ids)].groupby("ID").apply(
#     lambda df_for_ID: infer_and_plot_for_id(
#         df_for_ID, diff_threshold=1e-6
#     )
# )


ID: 101
Number of datapoints: 591
Pass 1 - Posteriors' diff for Healthy FEV1 (L): 1.1598247730123832
Pass 1 - Posteriors' diff for Healthy O2 saturation (%): 1.4626076335118903
Pass 2 - Posteriors' diff for Healthy FEV1 (L): 0.1822192562395705
Pass 2 - Posteriors' diff for Healthy O2 saturation (%): 3.612553237819037e-11

ID: 405
Number of datapoints: 1035
Pass 1 - Posteriors' diff for Healthy FEV1 (L): 1.1029346856259064
Pass 1 - Posteriors' diff for Healthy O2 saturation (%): 1.481342110311506
Pass 2 - Posteriors' diff for Healthy FEV1 (L): 0.2709656906997049
Pass 2 - Posteriors' diff for Healthy O2 saturation (%): 2.964134792857958e-12

ID: 272
Number of datapoints: 417
Pass 1 - Posteriors' diff for Healthy FEV1 (L): 1.3493082800195373
Pass 1 - Posteriors' diff for Healthy O2 saturation (%): 1.4714769810290058
Pass 2 - Posteriors' diff for Healthy FEV1 (L): 0.30242080708763597
Pass 2 - Posteriors' diff for Healthy O2 saturation (%): 3.3863207227479266e-11

ID: 201
Number of datapoi

KeyboardInterrupt: 

In [22]:
df_res_before_convergence.head()

,ID,Day,Age,Height,Sex,ecFEV1,ecFEF2575%ecFEV1,Healthy FEV1 (L),Healthy O2 saturation (%),Airway resistance (%),Inactive alveoli (%)
0,101,"1, 2019-01-25",53,173.0,Male,1.31,41.221374,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.7676005717732...","[1.301313441850872e-35, 8.428052859720377e-31,...","[3.2633986430657008e-09, 2.513419355016359e-08...","[0.5363556749812803, 0.2667678589813558, 0.113..."
1,101,"1, 2019-01-26",53,173.0,Male,1.31,43.511450,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8316610157043...","[1.301313441848166e-35, 8.428052859470501e-31,...","[8.003498293561854e-20, 9.049728942049443e-19,...","[0.5363556749812385, 0.2667678589826388, 0.113..."
2,101,"1, 2019-01-27",53,173.0,Male,1.31,51.145038,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.1711740278564...","[1.301313441833957e-35, 8.428052859445306e-31,...","[6.988551642309672e-34, 5.336074797061693e-32,...","[0.5363556749817887, 0.2667678589828187, 0.113..."
3,101,"1, 2019-01-28",53,173.0,Male,1.31,53.076923,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.7751201819768...","[1.3013134418254516e-35, 8.428052859451708e-31...","[2.124891438415337e-45, 6.752071129113207e-42,...","[0.536355674982094, 0.26676785898283173, 0.113..."
4,101,"1, 2019-01-29",53,173.0,Male,1.30,46.875000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.4588691072000...","[1.3013134418287196e-35, 8.428052859450828e-31...","[2.0189591764699957e-48, 5.158563715540386e-44...","[0.5363556749819469, 0.2667678589828348, 0.113..."


In [13]:
(
        _,
        inf_alg,
        HFEV1,
        ecFEV1,
        AR,
        HO2Sat,
        O2SatFFA,
        IA,
        UO2Sat,
        O2Sat,
        ecFEF2575prctecFEV1,
    ) = mb.o2sat_fev1_fef2575_long_model_shared_healthy_vars_and_temporal_ar(
        df.loc[0, "Height"], df.loc[0, "Age"], df.loc[0, "Sex"], ia_prior="breathe", ar_change_cpt_suffix=ar_change_cpt_suffix
    )

HFEV1_final, HO2Sat_final = shared_vars_final
first_m = HFEV1_final.vmessages['2019-01-25']

idx2 = 10
ar1 = df_res_before_convergence.loc[0, "Airway resistance (%)"]
ar2 = df_res_before_convergence.loc[idx2, "Airway resistance (%)"]

fev11 = df_res_before_convergence.loc[0, "Healthy FEV1 (L)"]
fev12 = df_res_before_convergence.loc[idx2, "Healthy FEV1 (L)"]

fig = make_subplots(
    rows=5, cols=1, vertical_spacing=0.02,
)
ih.plot_histogram(
    fig, AR, ar1, AR.a, AR.b, 1, 1, None, "#636EFA"
)
ih.plot_histogram(
    fig, AR, ar2, AR.a, AR.b, 2, 1, None, "#636EFA"
)
ih.plot_histogram(
    fig, HFEV1, HFEV1.cpt, HFEV1.a, HFEV1.b, 3, 1, None, "#EF553B"
)
m = first_m * HFEV1.cpt
m = m / np.sum(m)
ih.plot_histogram(
    fig, HFEV1, m, HFEV1.a, HFEV1.b, 4, 1, None, "#EF553B"
)
ih.plot_histogram(
    fig, HFEV1, fev11, HFEV1.a, HFEV1.b, 5, 1, None, "#EF553B"
)
fig.update_layout(
    title_text="AR and FEV1 posterior distribution",
    height=600,
    width=800,
)
fig.show()

In [5]:
df.iloc[0]

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,idx ecFEV1 (L),idx ecFEF25-75 % ecFEV1 (%),idx O2 saturation (%)
0,101,2019-01-25,1.31,97,0.54,1.31,0.67,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,99.845492,41.221374,26,20,47
1,101,2019-01-26,1.31,98,0.57,1.31,0.67,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,100.874827,43.511450,26,21,48
2,101,2019-01-27,1.31,96,0.67,1.31,0.69,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,98.816157,51.145038,26,25,46
3,101,2019-01-28,1.30,96,0.69,1.31,0.69,Male,173.0,53,3.610061,97.150104,36.287474,36.010470,98.816157,53.076923,26,26,46
4,101,2019-01-29,1.28,98,0.60,1.30,0.69,Male,173.0,53,3.610061,97.150104,36.010470,35.456463,100.874827,46.875000,26,23,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41255,553,2023-10-08,4.12,98,4.20,4.15,4.42,Male,179.0,18,4.619116,97.042483,89.844024,89.194549,100.986699,101.941748,83,50,48
41256,553,2023-10-11,4.15,97,4.06,4.15,4.84,Male,179.0,18,4.619116,97.042483,89.844024,89.844024,99.956223,97.831325,83,48,47
41257,553,2023-11-06,4.13,97,4.84,4.15,4.84,Male,179.0,18,4.619116,97.042483,89.844024,89.411041,99.956223,117.191283,83,58,47
41258,553,2023-11-08,4.05,96,4.71,4.16,4.99,Male,179.0,18,4.619116,97.042483,90.060516,87.679108,98.925746,116.296296,83,58,46


array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383, 0.0106383,
       0.0106383, 0.0106383, 0.0106383, 0.0106383, 

In [9]:
HFEV1.get_bins_arr()

array([[1.  , 1.05],
       [1.05, 1.1 ],
       [1.1 , 1.15],
       [1.15, 1.2 ],
       [1.2 , 1.25],
       [1.25, 1.3 ],
       [1.3 , 1.35],
       [1.35, 1.4 ],
       [1.4 , 1.45],
       [1.45, 1.5 ],
       [1.5 , 1.55],
       [1.55, 1.6 ],
       [1.6 , 1.65],
       [1.65, 1.7 ],
       [1.7 , 1.75],
       [1.75, 1.8 ],
       [1.8 , 1.85],
       [1.85, 1.9 ],
       [1.9 , 1.95],
       [1.95, 2.  ],
       [2.  , 2.05],
       [2.05, 2.1 ],
       [2.1 , 2.15],
       [2.15, 2.2 ],
       [2.2 , 2.25],
       [2.25, 2.3 ],
       [2.3 , 2.35],
       [2.35, 2.4 ],
       [2.4 , 2.45],
       [2.45, 2.5 ],
       [2.5 , 2.55],
       [2.55, 2.6 ],
       [2.6 , 2.65],
       [2.65, 2.7 ],
       [2.7 , 2.75],
       [2.75, 2.8 ],
       [2.8 , 2.85],
       [2.85, 2.9 ],
       [2.9 , 2.95],
       [2.95, 3.  ],
       [3.  , 3.05],
       [3.05, 3.1 ],
       [3.1 , 3.15],
       [3.15, 3.2 ],
       [3.2 , 3.25],
       [3.25, 3.3 ],
       [3.3 , 3.35],
       [3.35,

In [7]:
import src.data.helpers as dh

dftmp = dh.load_excel(
    f"{dh.get_path_to_main()}/ExcelFiles/BR/Refining_F3/infer_AR_with_two_days_model_O2Sat_FEV1.xlsx",
    # f"{dh.get_path_to_main()}/ExcelFiles/BR/Refining_F3/infer_AR_with_two_days_model_O2Sat_ecFEV1.xlsx",
    [AR.name],
    ["Day"],
).drop(columns=["Unnamed: 0", HO2Sat.name, IA.name, HFEV1.name])